In [ ]:
%%configure
{
  "defaultLakehouse": {
    "name": "YourLakehouseName",
    "id": "YourLakehouseId",
    "workspaceId": "YourWorkspaceId"
  }
}

In [ ]:

%pip install -U semantic-link-labs
from sempy_labs.tom import connect_semantic_model
import pandas as pd

csv_path = "/lakehouse/default/Files/relations_df.csv"

df = pd.read_csv(csv_path, dtype=str, sep=";").fillna("")
WORKSPACE = "YourWorkspaceName"
DATASET   = "YourSemanticModelName"



StatementMeta(, 5e8e4be3-8860-4849-9bcc-4b8c6089f993, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.9 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.7/764.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 53.8 MB/s eta 0:00:00


  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.30.2
    Not uninstalling azure-core at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-070b7be7-3f85-4d8a-b5fc-42fe90f8f0e0
    Can't uninstall 'azure-core'. No files were found to uninstall.
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.11.0
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-070b7be7-3f85-4d8a-b5fc-42fe90f8f0e0
    Can't uninstall 'semantic-link-sempy'. No files were found to uninstall.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fsspec-wrapper 0.1.15 requires PyJWT>=2.6.0, but you have pyjwt 2.4.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
print(df)

StatementMeta(, 5e8e4be3-8860-4849-9bcc-4b8c6089f993, 10, Finished, Available, Finished)

     from_table      to_table from_column to_column from_cardinality  \
0  dbo_CAPCPI01  dbo_CAPFRP02     NOPARTC   NOPARTC             Many   

  to_cardinality cross_filtering_behavior active  
0            One             OneDirection   True  


In [5]:
def normalize_bool(x:str) -> bool:
    return str(x).strip().lower() in {"true","1","yes","y","t"}

def rel_key(r):
    return (r.from_table.lower(), r.from_column.lower(),
            r.to_table.lower(),   r.to_column.lower())

created, skipped = 0, 0

with connect_semantic_model(dataset=DATASET,
                            workspace=WORKSPACE,
                            readonly=False) as model:
    attrs = [a for a in dir(model) if not a.startswith("_")]
    #print(attrs)
    relations = [a for a in dir(model.used_in_relationships) if not a.startswith("_")]
    #print(relations)
    # index existant – bien plus rapide que boucler sur model.relationships à chaque ligne
    existing = {rel_key(r) for r in relations}

    for _, row in df.iterrows():
        key = (row["from_table"].strip().lower(),
               row["from_column"].strip().lower(),
               row["to_table"].strip().lower(),
               row["to_column"].strip().lower())

        if key in existing:
            skipped += 1
            continue

        model.add_relationship(
            from_table=row["from_table"].strip(),
            from_column=row["from_column"].strip(),
            to_table=row["to_table"].strip(),
            to_column=row["to_column"].strip(),
            from_cardinality=row.get("from_cardinality","Many").strip(),
            to_cardinality=row.get("to_cardinality","One").strip(),
            cross_filtering_behavior=row.get("cross_filtering_behavior","SingleDirection").strip(),
            is_active=normalize_bool(row.get("active","True"))
        )
        created += 1

    #model.save_changes()

print(f"Relations créées : {created}, ignorées (déjà présentes) : {skipped}")

StatementMeta(, 5e8e4be3-8860-4849-9bcc-4b8c6089f993, 11, Finished, Available, Finished)

['add_calculated_column', 'add_calculated_table', 'add_calculated_table_column', 'add_calculation_group', 'add_calculation_item', 'add_changed_property', 'add_data_column', 'add_entity_partition', 'add_expression', 'add_field_parameter', 'add_hierarchy', 'add_incremental_refresh_policy', 'add_m_partition', 'add_measure', 'add_perspective', 'add_relationship', 'add_role', 'add_role_member', 'add_table', 'add_time_intelligence', 'add_to_perspective', 'add_translation', 'all_calculated_columns', 'all_calculated_tables', 'all_calculation_groups', 'all_calculation_items', 'all_columns', 'all_date_tables', 'all_hierarchies', 'all_hybrid_tables', 'all_levels', 'all_measures', 'all_partitions', 'all_rls', 'apply_refresh_policy', 'cardinality', 'clear_annotations', 'clear_extended_properties', 'clear_linguistic_schema', 'close', 'convert_direct_lake_to_import', 'copy_object', 'data_size', 'delete_synonym', 'depends_on', 'dictionary_size', 'format_dax', 'fully_qualified_measures', 'generate_meas